In [3]:
import numpy as np
from fluid import Fluid
from particles import Swarm
from utils import Vec
from levelset import Levelset

from debug import get_image
from tqdm import trange

from PIL import Image

import cProfile, pstats, io
from pstats import SortKey

# Size of domain in the X and Y direction
sX, sY = 1, 1 # meters

# Total time of the simulation
T = 1 # seconds

# Resolution of the domain discretization
w, h = 512, 512

# Space|Time step size of domain
dx = sX / w  # meters
dt = 0.01 * T # seconds

fls = np.ones((h, w)) * -1
sls = np.ones((h, w)) * +1

fls[0, :] = fls[-1, :] = fls[:, 0] = fls[:, -1] = +1
sls[0, :] = sls[-1, :] = sls[:, 0] = sls[:, -1] = -1

fls = Levelset(fls)
sls = Levelset(sls)

# Prepare the initial payload
ppc = 3

img = np.asarray(Image.open('painting.jfif').resize((w-2, h-2), resample = Image.NEAREST)).reshape(-1, 3)
pl = [{'ink' : Vec(float(r), float(g), float(b))} for r, g, b in img for _ in range(ppc)]

fluid = Fluid((w, h), fls, sls, ppc = ppc, pl = pl, dx = dx)

# init = Image.open('heart.jfif').resize((w, h), resample = Image.NEAREST).convert('L')
# init = Levelset(np.asarray(init, dtype = float) - 200)
# Vy = -1 * dx / dt
# print(Vy)

# for p in fluid.S.get(init): 
#     # NOTE: Velocity should be expressed in meter / second
#     p.update('vel', Vy, 'y') 
#     p.update('ink', Vec(255., 0., 0.))

initL = Image.open('start_left.jpeg' ).resize((w-2, h-2), resample = Image.NEAREST).convert('L')
initR = Image.open('start_right.jpeg').resize((w-2, h-2), resample = Image.NEAREST).convert('L')

initL = Levelset(np.asarray(initL, dtype = float) - 200)
initR = Levelset(np.asarray(initR, dtype = float) - 200)

PL = fluid.S.get(initL)
PR = fluid.S.get(initR)

for p in PL: p.update('vel', -1 * dx / dt, 'x')
for p in PR: p.update('vel', +1 * dx / dt, 'x')

IMG = [fluid.screenshot()]

# pr = cProfile.Profile()
# pr.enable()

steps = int(T / dt)
for _ in trange(steps, desc = 'Computing Fluid Simulation'):
    fluid.step(dt)

    IMG += [fluid.screenshot()]

# pr.disable()
# pr.print_stats(SortKey.CUMULATIVE)
IMG[0].save('debug.gif', save_all = True, append_images = IMG[1:], duration = 20, loop = 0)
# IMG[0].save('van_gogh_3.gif', save_all = True, append_images = IMG[1:], duration = 20, loop = 0)


Computing Fluid Simulation: 100%|██████████| 100/100 [5:08:04<00:00, 184.85s/it] 
